In [1]:
import os, sys, pandas as pd, numpy as np, string, json, math
from pandas.io.json import json_normalize
from random import randrange, shuffle

In [2]:
DATA_FOLDER = "../data/"

SYN_REPORTS_FOLDER = "../data/outputs/synthetic_reports/"

In [3]:
def create_metric_names(num_metrics = 100, prefixes = ["volume_sales", "value_sales", "arpu", "user_name", 
                                                       "phone_num", "usage_tier", "total_calls", "total_sms", 
                                                       "total_data", "monthly_plan", "subscriptions", 
                                                       "roaming_calls", "roaming_data", "roaming_sms", 
                                                       "volume sales", "value sales", "arpu", "user name", 
                                                       "phone num", "usage tier", "total calls", "total sms", 
                                                       "total data", "monthly plan", "subscriptions", 
                                                       "roaming calls", "roaming data", "roaming sms",
                                                      "sales(volume)", "value sales", "arpu", "user name", 
                                                       "phone num", "usage tier", "total calls", "total sms", 
                                                       "total data", "monthly plan", "subscriptions", 
                                                       "roaming calls", "data(roaming)", "roaming sms"]):
    
    """
    Creates metric names based on the prefixes provided.
    create num_metrics number of metrics totally
    """
    
    pre_len = len(prefixes)
    
    num_replica = (num_metrics//pre_len) + 1
    
    metrics_list = []
    
    for prefix in prefixes:
        
        for rep_num in range(num_replica):
            
            if len(metrics_list) >= num_metrics:
                
                break
            
            metrics_list.append(prefix + "_" + str(rep_num))
            
    
    return metrics_list
    
    

In [4]:
with open('../data/raw/telstra.json') as jsonfile:
    
    telstra = json.load(jsonfile)
    
telstra_df = json_normalize(telstra)

telstra_df.head()

/home/fractaluser/anaconda3/envs/telstra/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,text,tags,post_date,user,web_link,title
0,"[Not a bad deal., NOTE: Gift card can only be ...","[ Electrical & Electronics, Apple, Smartwatch]",on 08/11/2018 - 11:11,android iOS,ebay.com.au,Apple Watch Series 3 38mm Cellular $458.10 Del...
1,[Pretty much the same deal they have had long ...,"[ Mobile, Gift Card, Phone Plan, Telstra]",on 21/10/2019 - 20:19,ChiMot,jbhifi.com.au,Telstra 12-Month Post Paid 60GB $65 Per Month ...
2,"[Per email:, \nChristmas is a magical time of ...","[ Entertainment, Instagrampost, Movie Ticket]",on 07/11/2018 - 18:24,Bananaramas,telstra.com.au,10 Village/ Event Cinema Movie Vouchers for $9...
3,[You get a free $30 sim too in the deal. Prett...,"[ Mobile, Apple, Mobile Phone]",on 26/12/2018 - 13:17,swapsey,telstra.com.au,iPhone 6 32GB $329 Delivered @ Telstra (Online...
4,[Telstra has today unveiled its new $199 “Ulti...,"[ Mobile, Phone Plan]",on 24/07/2018 - 22:55,bonky,channelnews.com.au,$199 P/M for Unlimited Mobile Data/Calls/SMS/M...


In [5]:
def create_metric_desc(df, num_metrics = 100):
    
    """
    Takes a sample data frame that should have "title" as column and 
    values in these columns should be text
    
    Returns num_metrics no of list of texts 
    
    """
    
    text_lists = df["title"].tolist()
    
    total_texts = len(text_lists)
    
    repeats = math.ceil(num_metrics / total_texts )
    
    text_lists = text_lists * repeats
    
    
    
    return text_lists[:num_metrics]

In [6]:
NUM_METRICS = 500

metric_names = create_metric_names(NUM_METRICS)

print(len(metric_names))

metric_descriptions = create_metric_desc(telstra_df, num_metrics=NUM_METRICS)

print(len(metric_descriptions))

500
500


In [7]:
metrics_catalog_df = pd.DataFrame({"metrics": metric_names, "description": metric_descriptions})

metrics_catalog_df.sample(5)

,metrics,description
72,total_calls_0,Telstra Easy Control 102 Twin Cordless Phones ...
281,monthly plan_5,Apple Watch Series 3 Silver/Space Grey Dark Ol...
204,user name_0,Lifeproof iPhone 8 Plus FRE Case - Lime $25.00...
40,user_name_4,"[NSW, VIC] $500 Gift Card for Telstra Port-in ..."
317,roaming data_5,iPhone X Leather Folio Black $94.05 Delivered ...


In [8]:
metrics_catalog_df.to_csv("{}metrics_catalog_synthetic_v1_19feb20.csv".format(DATA_FOLDER + "outputs/"), index=False)

#### Reports Creation

In [9]:
column_names = create_metric_names(NUM_METRICS + 500, prefixes=["nsw", "qld", "sydney", "melbourne", 
                                                       "brisbane", "perth", "victoria", "canberra", 
                                                       "adelaid", "gold coast", "auckland"])

In [10]:
pd.options.display.max_rows = 1000

In [11]:
def _delete_files_from_folder(foldername = SYN_REPORTS_FOLDER):
    
    abs_path = os.path.abspath(SYN_REPORTS_FOLDER)
    
    print("Deleting files from {}".format(abs_path))
    
    filenames = os.listdir(abs_path)
    
    counter = 0
    
    for filename in filenames:
        
        abs_filename = os.path.join(abs_path, filename)

        try:
            os.remove(abs_filename)
            
            counter += 1
            
        except Exception as e:
            
            print("unable to delete "+ filename + "_" +str(e))
            
            pass
        
    print("{}/{} files deleted.".format(str(counter), str(len(filenames))))
    
    return

In [12]:
def create_reports(column_names, num_reports = 10, num_rows = 10):
    
    """
    Delete currently present reports, then create num_reports # of  excel files with num_rows
    
    Column names for eacg file will be randomly sampled from provided column names
    
    num cols per file is randomly varied to be around (len(column_names)//num_reports + 5)
    
    similarly number of rows are randomly varied for each report
    
    A string column is randomly added to each report. Values in this column will be randomly the column names themselves
    """
    
    num_columns = len(column_names)
    
    num_cols_rep = math.ceil(num_columns // num_reports) + 5
    
    reports_list = []
    
    """
    First delete all files in synthetic reports folder
    """
    _delete_files_from_folder(SYN_REPORTS_FOLDER)
    
    for rep in range(num_reports):
        
        num_columns_curr = randrange(2, num_cols_rep)
        
        string_col_num = randrange(1, num_columns_curr)
        
        num_rows_curr = randrange(2, num_rows + 5)
        
        col_nums = np.random.randint(1, num_columns, num_columns_curr).tolist()
        
        col_names = [column_names[x]for x in col_nums]
        
        string_colname = col_names[string_col_num]
        
        curr_report_df = pd.DataFrame(columns=col_names)
        
        for col in col_names:
            
            if col == string_colname:
                
                idxs = np.random.randint(1, num_columns, num_rows_curr).tolist()
                
                values = [column_names[x] for x in idxs]
                
                #print(col)
                
            else:
                
                values = np.random.rand(num_rows_curr).tolist()
            
            curr_report_df[col] = values
            
        print("{} report(s) created".format(str(rep + 1)))
        
        reports_list.append(curr_report_df)

        curr_report_df.to_excel("{}reports_synthetic_{}_v1_19feb20.xlsx".format(SYN_REPORTS_FOLDER, str(rep)), 
                                      index=False)
    
    return reports_list

In [13]:
df_list = create_reports(column_names, num_reports = 204, num_rows = 100)

Deleting files from /home/fractaluser/Narahari/Fractal/telstra_metric_buster/data/outputs/synthetic_reports
204/204 files deleted.
1 report(s) created
2 report(s) created
3 report(s) created
4 report(s) created
5 report(s) created
6 report(s) created
7 report(s) created
8 report(s) created
9 report(s) created
10 report(s) created
11 report(s) created
12 report(s) created
13 report(s) created
14 report(s) created
15 report(s) created
16 report(s) created
17 report(s) created
18 report(s) created
19 report(s) created
20 report(s) created
21 report(s) created
22 report(s) created
23 report(s) created
24 report(s) created
25 report(s) created
26 report(s) created
27 report(s) created
28 report(s) created
29 report(s) created
30 report(s) created
31 report(s) created
32 report(s) created
33 report(s) created
34 report(s) created
35 report(s) created
36 report(s) created
37 report(s) created
38 report(s) created
39 report(s) created
40 report(s) created
41 report(s) created
42 report(s) crea

In [14]:
def create_cols_metric_mapping(colnames, metric_names):
    
    """
    Creates a synthetic mapping b/w colnames and metric names
    
    Assumes, more than 1 column could be mapped to a metric
    """
    
    # Introducing duplicates in metrics
    metric_names_duplicated = metric_names * 2
    
    #Random shuffle
    shuffle(metric_names_duplicated)
    
    #Sample from this duplicated randomly shuffled metric names and assign them against columns
    
    num_cols = len(colnames)
    
    sampled_metric_names = metric_names_duplicated[:num_cols]
    
    col_met_mapping = pd.DataFrame({"column_names":colnames, "metric_names":sampled_metric_names})   
    
    
    col_met_mapping.to_csv("{}col_met_mapping_v1_19feb20.csv".format(DATA_FOLDER+"outputs/"), index=False)
    
    return col_met_mapping

In [15]:
col_met_mapping = create_cols_metric_mapping(colnames=column_names, metric_names=metric_names)

col_met_mapping.head(5)

,column_names,metric_names
0,nsw_0,phone num_11
1,nsw_1,phone_num_9
2,nsw_2,usage_tier_11
3,nsw_3,user name_3
4,nsw_4,roaming_calls_0


In [16]:
col_met_mapping.metric_names.duplicated().sum()

664

#### Users - Files -Owners mapping

In [17]:
names_list = telstra_df["user"].unique().tolist()

print(len(names_list))

names_list

610


['android iOS',
 'ChiMot',
 'Bananaramas',
 'swapsey',
 'bonky',
 'Dealma',
 'RtN',
 'save50',
 'pennypincher98',
 'readerr0r',
 'dealbot',
 'Tejas57',
 'JAY13',
 'Sanjup',
 'Amos',
 'deal junkie',
 'monty168',
 'nat why',
 'Aethelbert',
 'windowshopper',
 'ravendr',
 'SolidestVIDU',
 'bugger off',
 'Suju',
 'neilpatrickharris',
 'aiyoyo',
 'RYZEN',
 'Sergeant Salami',
 'unistudent1',
 'RogueWolf',
 'Suoflo',
 'Snowman1',
 'J4N9',
 'mooturner',
 'jv',
 'Nemali',
 'Tipu',
 'crazynic',
 'hek666',
 'jasonmurray',
 'rend586',
 'Oteta',
 'Dud',
 'kururii',
 'bozbargain',
 'kurdoxan',
 'jake772',
 'rocky7st',
 'jasswolf',
 'aaruu',
 'tbargain',
 'kokomo1978',
 'humbala',
 'Nalar',
 'kcbworth',
 'unknown101',
 'Nass1',
 'MikeKulls',
 'rba',
 'samsal46',
 'Cocococo111222333',
 'Ganda',
 'BooYa',
 'Yanluk',
 'downero',
 'singhdeals',
 'sc00bie',
 'lakmal',
 'seejay80',
 'bargain4all',
 'Hits',
 'Holdeningon',
 'quinteros',
 'mdsh',
 'wellzi',
 'Cheapnesssake',
 'jc89',
 'doweyy',
 'Snacky',
 'k

In [18]:
report_names = os.listdir(SYN_REPORTS_FOLDER)

In [19]:
def create_user_rep_owner_mapping(names_list, report_names):
    
    """
    Creates report - owners mapping
    and report - users mapping
    
    A owner can be mapped to multiple reports
    
    Each report will have multiple users in a many to many fashion
    """
    
    num_reports = len(report_names)
    
    num_names = len(names_list)
    
    #Duplicate usernames and randomly assign them to reports
    
    num_names_rep = math.ceil(num_reports / num_names) + 50
    
    names_list_dup = names_list * num_names_rep
    
    shuffle(names_list_dup)
    
    owner_names = names_list_dup[:num_reports]
    
    #Duplicate report names and randomly assign them to duplicated random users
    
    num_rep_rep = math.ceil(num_names / num_reports) + 3
    
    reports_list_dup = report_names * num_rep_rep
    
    shuffle(reports_list_dup)
    
    names_mult_factor = math.ceil(len(reports_list_dup) // num_names) + 1
    
    names_list_dup2 = names_list * names_mult_factor
    
    shuffle(names_list_dup2)
    
    user_names_list = names_list_dup2[: len(reports_list_dup)]
    
    owners_df = pd.DataFrame({"Reports": report_names, "Owners":owner_names})
    
    users_df = pd.DataFrame({"Reports": reports_list_dup, "Users":user_names_list})
    
    users_df.drop_duplicates(inplace=True)
    
    users_df.sort_values("Reports", inplace=True)
    
    owners_df.sort_values("Reports", inplace=True)
    
    users_df.reset_index(drop=True, inplace=True)
    
    print(users_df.shape)
    
    owners_df.to_csv("{}rep_owner_mapping_v1_19feb20.csv".format(DATA_FOLDER + "outputs/"), index=False)
    
    users_df.to_csv("{}rep_users_mapping_v1_19feb20.csv".format(DATA_FOLDER + "outputs/"), index=False)
    
    
    return [users_df, owners_df]

In [20]:
users_df, owners_df = create_user_rep_owner_mapping(names_list, report_names)

users_df

(1219, 2)


,Reports,Users
0,reports_synthetic_0_v1_19feb20.xlsx,rayboy2006
1,reports_synthetic_0_v1_19feb20.xlsx,interfreak
2,reports_synthetic_0_v1_19feb20.xlsx,Jase2801
3,reports_synthetic_0_v1_19feb20.xlsx,HAL
4,reports_synthetic_0_v1_19feb20.xlsx,Bargainaholic
...,...,...
1214,reports_synthetic_9_v1_19feb20.xlsx,luxx
1215,reports_synthetic_9_v1_19feb20.xlsx,sryanwib
1216,reports_synthetic_9_v1_19feb20.xlsx,Savas
1217,reports_synthetic_9_v1_19feb20.xlsx,TristanJ


In [21]:
print(owners_df.Owners.duplicated().sum())

33


In [22]:
owners_df

,Reports,Owners
99,reports_synthetic_0_v1_19feb20.xlsx,Suoflo
49,reports_synthetic_100_v1_19feb20.xlsx,PTG Benny
28,reports_synthetic_101_v1_19feb20.xlsx,sc00by
150,reports_synthetic_102_v1_19feb20.xlsx,paulj13
167,reports_synthetic_103_v1_19feb20.xlsx,McBanny
11,reports_synthetic_104_v1_19feb20.xlsx,iratepirate
103,reports_synthetic_105_v1_19feb20.xlsx,crazyj
153,reports_synthetic_106_v1_19feb20.xlsx,giles
164,reports_synthetic_107_v1_19feb20.xlsx,lastkey
12,reports_synthetic_108_v1_19feb20.xlsx,enayet
